In [1]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import config
import pprint
import pandas as pd
from bs4 import BeautifulSoup
import requests
import random
from time import sleep
#from random import randint

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

## Searching a playlist

## List of SPOTIFY Playlists

### Source 1

In [12]:
url_1 = "https://audiohype.io/resources/the-best-spotify-playlists/"
response_1 = requests.get(url_1)
print(response_1.status_code) # 200 status code means OK!
soup_1 = BeautifulSoup(response_1.content, "html.parser")

playlists_list_1 = [soup_1.select(".rank-math-link")[pl]["href"].split("?")[0] 
                    for pl in range(len(soup_1.select(".rank-math-link")))]
playlists_list_1 = [url.split("/")[-1] for url in playlists_list_1][:-1]
print(f"List of {len(playlists_list_1)} playlists has been created!")

200
List of 28 playlists has been created!


### Source 2

In [13]:
url_2 = "https://www.noteburner.com/spotify-music-tips/top-10-spotify-playlist.html"
response_2 = requests.get(url_2)
print(response_2.status_code) # 200 status code means OK!
soup_2 = BeautifulSoup(response_2.content, "html.parser")

playlists_list_2 = [soup_2.select("div.contaienr > div > div > ul > li > a")[pl]["href"].split("?")[0] 
                    for pl in range(len(soup_2.select("div.contaienr > div > div > ul > li > a")))]
playlists_list_2 = [url.split("/")[-1] for url in playlists_list_2][:-1]
print(f"List of {len(playlists_list_2)} playlists has been created!")

200
List of 9 playlists has been created!


### Create complete list of all playlists_lists

In [14]:
playlists_list = []
playlists_list.extend(playlists_list_1)
playlists_list.extend(playlists_list_2)
print(f"Total number of playlist IDs is {len(playlists_list)}")

Total number of playlist IDs is 37


### Add additional playlists

In [15]:
theSoundOfEverything = "69fEt9DN5r4JQATi52sRtq"
peacefulPiano = "37i9dQZF1DX4sWSpwq3LiO"
rockSolid = "37i9dQZF1DX49jUV2NfGku"
rockHard = "37i9dQZF1DWWJOmJ7nRx0C"
nuMetalEra = "37i9dQZF1DXcfZ6moR6J0G"
brainFood = "37i9dQZF1DWXLeA8Omikj7"
danceClassics = "37i9dQZF1DX8a1tdzq5tbM"
hype = "37i9dQZF1DX4eRPd9frC1m"
dubstepDon = "37i9dQZF1DX5Q27plkaOQ3"
thisIsBeethoven = "37i9dQZF1DX12YIlzNkccL"
playlists_list.append(theSoundOfEverything)
playlists_list.append(peacefulPiano)
playlists_list.append(rockSolid)
playlists_list.append(rockHard)
playlists_list.append(nuMetalEra)
playlists_list.append(brainFood)
playlists_list.append(danceClassics)
playlists_list.append(hype)
playlists_list.append(dubstepDon) 
playlists_list.append(thisIsBeethoven) 
print(f"Total number of playlist IDs is {len(playlists_list)}")

Total number of playlist IDs is 47


## Playlist functions

### Extracting `<TRACKS>`

In [ ]:
def get_playlist_tracks(username, playlists_list):

    tracks_base = []

    for playlist in playlists_list:

        results = sp.user_playlist_tracks(username,playlist,market="DE")
        tracks = results['items']
        while results['next']:
            results = sp.next(results)
            tracks.extend(results['items'])
            tracks_base.extend(tracks)
    print(f"{len(tracks_base)} tracks extracted!")  
    #tracks_base = list(filter(None, tracks_base[0]))
    return tracks_base

playlist=get_playlist_tracks("spotify", playlists_list)

### Extracting song `<IDs>`

In [8]:
def get_playlist_track_ids(playlist):
    
    song_ids_list = []
    
    for id in range(len(playlist)):
        try:
            song_ids_list.append(playlist[id]["track"]["id"])
        except:
            pass
    print(f"{len(set(song_ids_list))} track IDs have been extracted!")    
    return list(set(song_ids_list))

song_ids_list = get_playlist_track_ids(playlist)

10609 track IDs have been extracted!


### Extracting song `<AUDIO FEATURES>`

In [9]:
#df_base = pd.DataFrame(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       #'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       #'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       #'time_signature'])

In [ ]:
sleep(random.randint(0,3))

In [9]:
df_base = pd.DataFrame()

lower_bound_iterables = list(range(0, len(song_ids_list), 1000))
upper_bound_iterables = list(range(1000, len(song_ids_list), 1000))
upper_bound_iterables.append(len(song_ids_list))

csv_name_list = ["Batch" + str(batch+1) + ".csv" for batch in range(len(lower_bound_iterables))]

for idx in range(len(lower_bound_iterables)):
    list_of_audio_features=[]
    print(f"Sleeping for {sleep(random.randint(0,3))} seconds...")
    for item in range(lower_bound_iterables[idx],upper_bound_iterables[idx]):
        if item % 100 == 0:
            print(f"Working on {item}...")
        
        list_of_audio_features.append(sp.audio_features(song_ids_list[item])[0])
        list_of_audio_features = list(filter(None, list_of_audio_features))

    df=pd.DataFrame(list_of_audio_features)    
    df=df[["danceability","energy","loudness","speechiness","acousticness",
        "instrumentalness","liveness","valence","tempo","id","duration_ms"]]
    df.to_csv(csv_name_list[idx], index=False)
    df_base = pd.concat([df_base, df], ignore_index=True)
    #df_base.append(df, ignore_index=True)

Working on 0...
Sleeping for None seconds...
Sleeping for None seconds...
Sleeping for None seconds...
Sleeping for None seconds...
Sleeping for None seconds...
Sleeping for None seconds...
Sleeping for None seconds...
Sleeping for None seconds...
Sleeping for None seconds...
Sleeping for None seconds...
Sleeping for None seconds...
Sleeping for None seconds...
Sleeping for None seconds...


KeyboardInterrupt: 

In [60]:
df_base

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms
0,0.750,0.82100,-6.504,0.0437,0.13600,0.050300,0.3250,0.6540,130.015,3oHOObxpYRdsRwVqizxWNM,159562
1,0.756,0.96200,-7.526,0.0583,0.06710,0.703000,0.1940,0.7490,136.981,45bfH0GZvUyujIBiKRhXso,358267
2,0.546,0.87000,-6.795,0.0384,0.00362,0.659000,0.0527,0.0360,129.998,4BG4Smwt9cu1RzAKurrIN6,373167
3,0.705,0.88000,-8.887,0.0631,0.00481,0.862000,0.0512,0.5810,127.982,3AaGCSpUFAmtB1XGxz5IWk,245208
4,0.576,0.84800,-5.935,0.0333,0.00161,0.001440,0.1100,0.5320,130.026,2edB8M0PPZCoYKm8ohDaNm,277433
...,...,...,...,...,...,...,...,...,...,...,...
2984,0.384,0.00002,-45.272,0.5060,0.96900,0.939000,0.1250,0.0000,129.754,1JppKb6Em7CgtEBcjx0xt4,336246
2985,0.145,0.25600,-22.702,0.0392,0.95200,0.959000,0.0955,0.0543,138.581,0hX5RmR6yAjuHHC414n28y,865155
2986,0.603,0.49600,-8.555,0.0271,0.29300,0.000921,0.0705,0.0564,126.985,0tlnLS1RyUOXKCtFI0yF5n,248882
2987,0.310,0.50900,-10.122,0.0331,0.00137,0.955000,0.0981,0.2230,99.267,1cdb4cAbn3JwK6eRexqSQp,224652


### Extracting song `<NAMES>`

In [7]:
def get_playlist_track_names(playlist):
    song_names_list = []
    for t in range(len(playlist)):
        try:
            song_names_list.append(playlist[t]["track"]["name"])
        except:
            pass
    #song_names_list = [playlist[t]["track"]["name"] for t in range(len(playlist))]
    return song_names_list

song_names_list = get_playlist_track_names(playlist)

### Extracting song `<ARTISTS>`

In [8]:
def get_playlist_track_artists(playlist):
    song_artist_list = []
    for artist in range(len(playlist)):
        try:
            song_artist_list.append(playlist[artist]["track"]["artists"][0]["name"])
        except:
            pass
    #song_artist_list = [playlist[artist]["track"]["artists"][0]["name"] for artist in range(len(playlist))]
    return song_artist_list

song_artist_list = get_playlist_track_artists(playlist)

## Create dataframe with song `<NAMEs>`, `<ARTISTs>`, `<IDs>`, `<AUDIO FEATUREs>`

In [15]:
def create_df_to_concat_to_base():

    df_1 = pd.DataFrame(data={
        "song_name": song_names_list,
        "song_artist": song_artist_list,
        "id": song_ids_list
    })

    #df_2 = pd.DataFrame(song_audiofeatures_list)    
    #df_2 = df_2[["id","danceability","energy","loudness","speechiness","acousticness",
               #"instrumentalness","liveness","valence","tempo","duration_ms"]]

    #df = pd.concat([df_1, df_2], axis=1)
    #print(f"Dataframe with {len(df)} entries was created as `df_to_concat`")
    return df_1

df = create_df_to_concat_to_base()

In [16]:
df.to_csv("Song_Infos.csv", index=False)

## Extend csv with dataframe to concat

In [12]:
def extend_collection():
    
    df_base = pd.read_csv("Song_Collection_Extended.csv")
    df_extended = pd.concat([df_base, df], axis=0, ignore_index=True)
    df_extended.to_csv("Song_Collection_Extended.csv", index=False)
    print(f"Extended base dataframe (length = {len(df_base)}) by {len(df)} songs!")
    return df_extended

#df_extended = extend_collection()

### Create very first dataframe to build upon

In [13]:
def create_base_dataframe():

    df_1 = pd.DataFrame(data={
        "song_name": song_names_list,
        "song_artist": song_artist_list,
    })

    df_2 = pd.DataFrame(song_audiofeatures_list)    
    df_2 = df_2[["id","danceability","energy","loudness","speechiness","acousticness",
               "instrumentalness","liveness","valence","tempo","duration_ms"]]

    df = pd.concat([df_1, df_2], axis=1)
    df.to_csv("Song_Collection_Extended.csv", index=False)
    print(f"Dataframe with {len(df)} entries was created")
    return df

## PIPELINE - Looping through list of playlists